In [1]:
# Imports 

# Global Parameters
filename = "./pandemic_data.txt"
beta_1, delta_1 = 0.01, 0.6
num_vaccines_1 = range(20, 30, 1)
beta_2, delta_2 = 0.2, 0.7
num_vaccines_2 = range(15, 50, 5)

In [2]:
class Pandemic:
    
    # Initialize required parameter
    def __init__(self, b, d, num_vaccine):
        self.beta = b             # Transmission prob
        self.delta = d             # Healing prob
        self.num_vaccines = num_vaccine